In [1]:
import warnings
warnings.filterwarnings('ignore')
import os

import tws_data 
import statsmodels.tsa.stattools as ts
import statsmodels.api as sm
from ib_insync import *
import numpy as np
import pandas as pd
import datetime as dt
import time
pd.options.display.float_format = '{:,.4f}'.format
%matplotlib inline
from scipy.stats import norm as norm
import matplotlib.pyplot as plt
import requests as req
from bs4 import BeautifulSoup as bs
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import math


In [2]:
os.chdir('D:\Options Data\IB Intraday')

def create_dfrow(curr_option):
    df_row = pd.DataFrame({'Symbol': curr_option.contract.localSymbol, 
                           'Type':curr_option.contract.right, 
                           'Bid': curr_option.bid, 
                           'Ask': curr_option.ask, 
                           'Volume': curr_option.volume, 
                           'Strike': curr_option.contract.strike,
                           'Expiry': dt.datetime.strptime(curr_option.contract.lastTradeDateOrContractMonth, '%Y%m%d'),
                           'bidIV': curr_option.bidGreeks.impliedVol, 
                           'bidDelta': curr_option.bidGreeks.delta,
                           'bidGamma': curr_option.bidGreeks.gamma, 
                           'bidVega': curr_option.bidGreeks.vega,
                           'bidTheta': curr_option.bidGreeks.theta, 
                           'askIV': curr_option.askGreeks.impliedVol, 
                           'askDelta': curr_option.askGreeks.delta,
                           'askGamma': curr_option.askGreeks.gamma, 
                           'askVega': curr_option.askGreeks.vega,
                           'askTheta': curr_option.askGreeks.theta,
                           'modelIV': curr_option.modelGreeks.impliedVol, 
                           'modelDelta': curr_option.modelGreeks.delta,
                           'modelGamma': curr_option.modelGreeks.gamma, 
                           'modelVega': curr_option.modelGreeks.vega,
                           'modelTheta': curr_option.modelGreeks.theta}, index = [0])
    return df_row

def get_movers():

    indexarb_url = 'http://www.indexarb.com/stockVsIndexesMiddleColumn.html#djIndexAnalysis'
    indexarb_soup = bs(req.get(indexarb_url).text,'lxml')
    indexarb_tables = indexarb_soup.select('table[align="LEFT"]')

    movers = []

    for i, table_html in enumerate(indexarb_tables):
        curr_table = pd.read_html(str(table_html))[2]
        curr_table.columns = curr_table.loc[1,:].tolist()

        if i == 0:
            curr_table = curr_table.iloc[2:,:4].set_index('Index').apply(pd.to_numeric)
        else:
            curr_table = curr_table.iloc[2:,[1,3,4,5,6]].dropna().set_index('Sym').apply(pd.to_numeric)

        curr_table.columns = [x.split(' ')[0] for x in curr_table.columns.tolist()]
        movers.append(curr_table)

    index = movers[0]
    movers = pd.concat(movers[1:], axis = 0)
    movers = movers.sort_values('OneMonth')
    
    return index, movers

def time_remaining(weekly_options, curr_spx):
    typ = weekly_options.reset_index().loc[0,'Type']

    try:
        interest_rate = bs(req.get('https://www.marketwatch.com/investing/bond/tmubmusd01m?countrycode=bx','lxml').text, 'lxml')
        interest_rate = float(interest_rate.select_one('h3[class*="intraday__price"]').select_one('bg-quote').text)/100
    except:
        interest_rate = 0.02422

    weekly_options['a'] = interest_rate - (weekly_options['modelIV']**2)/2

    if typ == 'P':
        weekly_options['b'] = weekly_options['modelIV']*norm.ppf(-weekly_options['modelDelta'])
    if typ == 'C':
        weekly_options['b'] = -weekly_options['modelIV']*norm.ppf(weekly_options['modelDelta'])
    weekly_options['c'] = np.log(curr_spx/weekly_options['Strike'])

    weekly_options_otm = weekly_options[abs(weekly_options.modelDelta) <= 0.5]
    weekly_options_itm = weekly_options[abs(weekly_options.modelDelta) > 0.5]

    if typ == 'P':
        weekly_options_otm['time_remaining'] = ((-weekly_options_otm.b - 
                                                 np.sqrt(weekly_options_otm.b**2 - 
                                                         4*weekly_options_otm.a*weekly_options_otm.c))/(2*weekly_options_otm.a))**2
        weekly_options_itm['time_remaining'] = ((-weekly_options_itm.b + 
                                                 np.sqrt(weekly_options_itm.b**2 - 
                                                         4*weekly_options_itm.a*weekly_options_itm.c))/(2*weekly_options_itm.a))**2
    if typ == 'C':
        weekly_options_otm['time_remaining'] = ((-weekly_options_otm.b + 
                                                 np.sqrt(weekly_options_otm.b**2 - 
                                                         4*weekly_options_otm.a*weekly_options_otm.c))/(2*weekly_options_otm.a))**2
        weekly_options_itm['time_remaining'] = ((-weekly_options_itm.b - 
                                                 np.sqrt(weekly_options_itm.b**2 - 
                                                         4*weekly_options_itm.a*weekly_options_itm.c))/(2*weekly_options_itm.a))**2
    weekly_times = pd.concat([weekly_options_otm, 
                              weekly_options_itm], axis = 0)[['Expiry','time_remaining']].dropna().groupby('Expiry').mean()

    del weekly_options['a'], weekly_options['b'], weekly_options['c'], weekly_options_itm, weekly_options_otm

    return weekly_options.merge(weekly_times.reset_index(), on = 'Expiry')


def get_options(ticker, exchange, contractType, earliest_date, latest_date, curr_iv, stdevs, save_csv = False):
    if contractType == 'Stock':
        security = Stock(ticker,exchange)
    else:
        security = Index(ticker, exchange)

    ib.qualifyContracts(security)

    chains = ib.reqSecDefOptParams(security.symbol, '', security.secType, security.conId)

    chains_df = util.df(chains)
    chain = next(c for c in chains if c.tradingClass == ticker and c.exchange == 'SMART')

    all_expirations = sorted(exp for exp in chain.expirations)
    expirations_dates = [dt.datetime.strptime(x, '%Y%m%d') for x in all_expirations]

    expirations = list(filter(lambda x: x[0] >= earliest_date and x[0] <= latest_date, list(zip(expirations_dates,all_expirations))))
    dte_expirations = [(expir[0] - dt.datetime.today()).days for expir in expirations]
    expirations = [expir[1] for expir in expirations]

    ib.reqHeadTimeStamp(security, whatToShow='TRADES', useRTH=True)

    end_date = '' #'20100506 13:00:00 PST'
    duration = '1 D'

    sec_bars = ib.reqHistoricalData(
            security,
            endDateTime=end_date,
            durationStr=duration,
            barSizeSetting='1 min',
            whatToShow='TRADES',
            useRTH=True,
            formatDate=1)


    sec_df = util.df(sec_bars).set_index('date')
    sec_df.index = pd.to_datetime(sec_df.index)

    curr_sec_price = sec_df.reset_index().loc[len(sec_df) - 1, 'close']
    bounds_lst = [curr_sec_price*(stdevs*stdevs/(100*np.sqrt(365.25/dte))) for dte in dte_expirations]


    contracts_list = []

    for bounds, expiration in zip(bounds_lst, expirations):
#         strikes = [strike for strike in chain.strikes
#                    if strike % 5 == 0
#                    and curr_sec_price - bounds < strike < curr_sec_price + bounds]
        strikes = [strike for strike in chain.strikes
                   if curr_sec_price - bounds < strike < curr_sec_price + bounds]

        rights = ['P', 'C']

        curr_contracts = [Option(ticker, expiration, strike, right, 'SMART')
                          for right in rights
                          for strike in strikes]
        contracts_list += [curr_contracts]

    puts_list = []
    calls_list = []

    for contracts in contracts_list:
        ib.qualifyContracts(*contracts)
        sec_options = ib.reqTickers(*contracts)

        puts_rows = []
        calls_rows = []

        for curr_option in sec_options:
            try:
                curr_row = create_dfrow(curr_option)
                if curr_option.contract.right == 'P':
                    puts_rows.append(curr_row)
                elif curr_option.contract.right == 'C':
                    calls_rows.append(curr_row)
            except:
                continue

        puts = pd.concat(puts_rows).reset_index(drop = True).sort_values(['Expiry','Strike'])
        calls = pd.concat(calls_rows).reset_index(drop = True).sort_values(['Expiry','Strike'])
        puts = time_remaining(puts, curr_sec_price)
        calls = time_remaining(calls, curr_sec_price)
        puts_list.append(puts)
        calls_list.append(calls)
        time.sleep(5)

    puts_df = pd.concat(puts_list,axis = 0).reset_index(drop = True)
    calls_df = pd.concat(calls_list,axis = 0).reset_index(drop = True)

    if save_csv:
        curr_time = dt.datetime.now().strftime('%Y%m%d-%H-%M')
        puts_df.to_csv('ib_puts' + curr_time + '.csv')
        calls_df.to_csv('ib_calls' + curr_time + '.csv')
        
    return puts_df, calls_df, curr_sec_price


def get_spreads(weekly_options, curr_spx, expiry_index = 0, rowshift = 1, commissions = 0.0266):

    expirations_dates = weekly_options.Expiry.drop_duplicates().tolist()

    weekly_options = weekly_options[weekly_options.Expiry == expirations_dates[expiry_index]]

    typ = weekly_options.reset_index().loc[0,'Type']

    if typ == 'P':
        weekly_options = weekly_options.sort_values('Strike', ascending = False).reset_index(drop = True)
    if typ == 'C':
        weekly_options = weekly_options.sort_values('Strike', ascending = True).reset_index(drop = True)

    time_remaining = weekly_options.reset_index(drop = True).loc[0,'time_remaining']

    weekly_options['Mid'] = (weekly_options.Ask + weekly_options.Bid)/2
    shorts = weekly_options[['Mid', 'Strike', 'bidDelta', 'bidGamma', 'bidTheta', 'bidVega']]
    shorts.columns = ['Price','Strike','Delta','Gamma','Theta','Vega']
    longs = weekly_options[['Mid','Strike', 'askDelta', 'askGamma','askTheta', 'askVega']]
    longs.columns = ['Price','Strike','Delta','Gamma','Theta','Vega']

    spreads = shorts - longs.shift(-rowshift)
    spreads['Short_Strike'] = shorts.Strike
    spreads['Short_Delta'] = shorts.Delta
    spreads['Long_Strike'] = longs.Strike.shift(-rowshift)
    spreads = spreads[['Short_Strike', 'Long_Strike', 'Short_Delta', 'Price',
                       'Strike', 'Delta', 'Gamma', 'Theta', 'Vega']]

    spreads['Credit'] = spreads.Price - commissions

    if typ == 'P':
        spreads['MaxLoss'] = (-spreads.Strike + spreads.Credit)*100
        spreads['BreakEven'] = spreads['Short_Strike'] - spreads.Credit
    if typ == 'C':
        spreads['MaxLoss'] = (spreads.Strike + spreads.Credit)*100
        spreads['BreakEven'] = spreads['Short_Strike'] + spreads.Credit

    del spreads['Price'], spreads['Strike']

    spreads = spreads[spreads.Credit > 0].dropna().reset_index(drop = True)
    spreads['Expiry'] = expirations_dates[expiry_index]

    return spreads

def art_hedges(curr_calls, curr_puts, curr_price, expiry_index, bias = 'upside',
               maxshift = 6, commissions = 0.0266):
    
    hedge = []
    for rowshift in range(1,maxshift):

        if bias == 'upside':
            spreads = get_spreads(curr_puts, curr_price, expiry_index, rowshift, commissions = 0.0266)
        else:
            spreads = get_spreads(curr_calls, curr_price, expiry_index, rowshift, commissions = 0.0266)
        spreads = spreads[(spreads.Credit > 0.5)]
        hedge.append(spreads)

    hedge = pd.concat(hedge, axis = 0).reset_index(drop = True)

    if bias == 'upside':
        expirations_dates = curr_calls.Expiry.drop_duplicates().tolist()

        long_hedge = curr_calls[(curr_calls.Expiry == expirations_dates[expiry_index])][['Ask', 'Bid', 'Expiry', 'Strike',
                                                                             'modelDelta', 'modelGamma',
                                                                             'modelTheta', 'modelVega', 'time_remaining']].reset_index(drop = True)
    else:
        expirations_dates = curr_puts.Expiry.drop_duplicates().tolist()

        long_hedge = curr_puts[(curr_puts.Expiry == expirations_dates[expiry_index])][['Ask', 'Bid', 'Expiry', 'Strike',
                                                                             'modelDelta', 'modelGamma',
                                                                             'modelTheta', 'modelVega', 'time_remaining']].reset_index(drop = True)

    long_hedge['Mid'] = (long_hedge['Ask'] + long_hedge['Bid'])/2
    long_hedge.columns = [x.replace('model','') for x in long_hedge.columns.tolist()]

    hedges_lst = []

    for idx, row in hedge.iterrows():

        curr_hedge = long_hedge[long_hedge.Mid < row.Credit][['Mid','Expiry', 'Strike',
                                                              'Delta', 'Gamma',
                                                              'Theta', 'Vega', 'time_remaining']]
        curr_hedge['Gamma'] = curr_hedge['Gamma'] - row.Gamma
        curr_hedge['Delta'] = curr_hedge['Delta'] - row.Delta
        curr_hedge['Theta'] = curr_hedge['Theta'] - row.Theta
        curr_hedge['Vega'] = curr_hedge['Vega'] - row.Vega
        if bias == 'upside':
            curr_hedge = curr_hedge[(curr_hedge.Delta > 0) &
                                    (curr_hedge.Gamma > 0) &
                                    (curr_hedge.Theta > 0) &
                                    (curr_hedge.Vega > 0)]
        else:
            curr_hedge = curr_hedge[(curr_hedge.Delta < 0) &
                                (curr_hedge.Gamma > 0) &
                                (curr_hedge.Theta > 0) &
                                (curr_hedge.Vega > 0)]
        curr_hedge['Credit'] = -curr_hedge.Mid + row.Credit - 0.0133

        if bias == 'upside':
            curr_hedge['MaxLoss'] = (row.Long_Strike - row.Short_Strike + curr_hedge.Credit)*100
            curr_hedge['BreakEven'] = row.Short_Strike - curr_hedge['Credit']
        else:
            curr_hedge['MaxLoss'] = (row.Short_Strike - row.Long_Strike + curr_hedge.Credit)*100
            curr_hedge['BreakEven'] = row.Short_Strike + curr_hedge['Credit']
        curr_hedge['Maturity'] = curr_hedge['Expiry']
        curr_hedge['Mid'] = row.Short_Strike
        curr_hedge['Expiry'] = row.Long_Strike

        if bias == 'upside':
            curr_hedge.columns = ['Short_Put_Strike','Long_Put_Strike','Long_Call_Strike'] + curr_hedge.columns.tolist()[3:]
        else:
            curr_hedge.columns = ['Short_Call_Strike','Long_Call_Strike','Long_Put_Strike'] + curr_hedge.columns.tolist()[3:]

        hedges_lst.append(curr_hedge)

    hedges_df = pd.concat(hedges_lst, axis = 0)
    
    return hedges_df

In [3]:
index, movers = get_movers()

In [4]:
losers = movers.head()
winners = movers.tail().sort_values('OneMonth', ascending = False)

In [4]:
client_id = 13

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=client_id)

<IB connected to 127.0.0.1:7496 clientId=13>

In [4]:
# exchange = 'NYSE'
# contractType = 'Stock'
# earliest_date = dt.datetime(2019,4,6)
# latest_date = dt.datetime(2019,6,30)
# curr_iv = 40
# stdevs = 9

# stock_prices = {}
# stock_puts = {}
# stock_calls = {}

# for ticker in losers.index:

#     curr_puts, curr_calls, curr_price = get_options(ticker, exchange, contractType, earliest_date, latest_date, curr_iv, stdevs, save_csv = False)
    
#     stock_prices[ticker] = curr_price
#     stock_puts[ticker] = curr_puts
#     stock_calls[ticker] = curr_calls
#     break

In [ ]:
exchange = 'CBOE'
contractType = 'Index'
earliest_date = dt.datetime(2020,9,19)
latest_date = dt.datetime(2020,10,16)

ticker = 'SPX'
curr_iv = 30
stdevs = 30
curr_puts, curr_calls, curr_price = get_options(ticker, exchange, contractType, earliest_date, latest_date, curr_iv, stdevs, save_csv = False)

Started to throttle requests
Error 200, reqId 1601: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20201015', strike=2060.0, right='P', exchange='SMART')
Error 200, reqId 1602: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20201015', strike=2070.0, right='P', exchange='SMART')
Error 200, reqId 1605: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20201015', strike=2090.0, right='P', exchange='SMART')
Error 200, reqId 1604: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20201015', strike=2080.0, right='P', exchange='SMART')
Error 200, reqId 1607: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20201015', strike=2110.0, right='P', exchange='SMART')
E

Error 200, reqId 1658: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20201015', strike=2370.0, right='P', exchange='SMART')
Error 200, reqId 1660: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20201015', strike=2380.0, right='P', exchange='SMART')
Error 200, reqId 1661: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20201015', strike=2385.0, right='P', exchange='SMART')
Error 200, reqId 1663: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20201015', strike=2395.0, right='P', exchange='SMART')
Error 200, reqId 1662: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20201015', strike=2390.0, right='P', exchange='SMART')
Error 200, reqId 1666: No secu

Error 200, reqId 1712: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20201015', strike=2620.0, right='P', exchange='SMART')
Error 200, reqId 1710: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20201015', strike=2610.0, right='P', exchange='SMART')
Error 200, reqId 1715: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20201015', strike=2635.0, right='P', exchange='SMART')
Error 200, reqId 1714: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20201015', strike=2630.0, right='P', exchange='SMART')
Error 200, reqId 1716: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20201015', strike=2640.0, right='P', exchange='SMART')
Error 200, reqId 1718: No secu

Error 200, reqId 1763: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20201015', strike=2845.0, right='P', exchange='SMART')
Error 200, reqId 1764: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20201015', strike=800.0, right='P', exchange='SMART')
Error 200, reqId 1766: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20201015', strike=4900.0, right='P', exchange='SMART')
Error 200, reqId 1767: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20201015', strike=2855.0, right='P', exchange='SMART')
Error 200, reqId 1769: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20201015', strike=2865.0, right='P', exchange='SMART')
Error 200, reqId 1768: No secur

Error 200, reqId 1814: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20201015', strike=3070.0, right='P', exchange='SMART')
Error 200, reqId 1816: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20201015', strike=3080.0, right='P', exchange='SMART')
Error 200, reqId 1817: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20201015', strike=3085.0, right='P', exchange='SMART')
Error 200, reqId 1818: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20201015', strike=3090.0, right='P', exchange='SMART')
Error 200, reqId 1819: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20201015', strike=3095.0, right='P', exchange='SMART')
Error 200, reqId 1820: No secu

In [ ]:
maxshift = 50

arteHedges_upside = []
for expiry_index in range(len(curr_calls.Expiry.drop_duplicates())):
    bias = 'upside'
    possible_pos = art_hedges(curr_calls, curr_puts, curr_price, expiry_index, bias,
                              maxshift = maxshift, commissions = 0.0266)
    arteHedges_upside.append(possible_pos)

arteHedges_upside = pd.concat(arteHedges_upside,axis = 0)

In [ ]:
arteHedges_downside = []
for expiry_index in range(len(curr_calls.Expiry.drop_duplicates())):
    bias = 'downside'
    possible_pos = art_hedges(curr_calls, curr_puts, curr_price, expiry_index, bias,
                              maxshift = maxshift, commissions = 0.0266)
    arteHedges_downside.append(possible_pos)
arteHedges_downside = pd.concat(arteHedges_downside,axis = 0)

arteHedges_upside['ReturnOnRisk'] = abs(arteHedges_upside.Credit/arteHedges_upside.MaxLoss)
arteHedges_downside['ReturnOnRisk'] = abs(arteHedges_downside.Credit/arteHedges_downside.MaxLoss)

In [ ]:
potential_ups = arteHedges_upside[(arteHedges_upside.Short_Put_Strike < curr_price) &
                                  (arteHedges_upside.Short_Put_Strike > 0.001)].sort_values(['Maturity', 
                                                                                             'ReturnOnRisk'], ascending = False)
try:
    potential_ups['downmove_to_breach'] = potential_ups['Short_Put_Strike']/curr_price - 1
    potential_ups['upmove_to_win'] = potential_ups['Long_Call_Strike']/curr_price - 1
    potential_ups = potential_ups.sort_values('MaxLoss').reset_index(drop = True)
    potential_ups['MaxLossScore'] = potential_ups.index
    
    potential_ups = potential_ups.sort_values('downmove_to_breach', ascending = False).reset_index(drop = True)
    potential_ups['DownMoveScore'] = potential_ups.index
    
    potential_ups = potential_ups.sort_values('upmove_to_win', ascending = False).reset_index(drop = True)
    potential_ups['UpMoveScore'] = potential_ups.index
    
    potential_ups['Score'] = potential_ups.MaxLossScore + potential_ups.DownMoveScore + potential_ups.UpMoveScore
    potential_ups = potential_ups.sort_values('Score', ascending = False)
except:
    None
potential_ups

In [ ]:
potential_downs = arteHedges_downside[(arteHedges_downside.Short_Call_Strike > curr_price)].sort_values(['Maturity', 
                                                                                                         'Long_Put_Strike'], ascending = False)
try:
    potential_downs['upmove_to_breach'] = potential_downs['Short_Call_Strike']/curr_price - 1
    potential_downs['downmove_to_win'] = potential_downs['Long_Put_Strike']/curr_price - 1
    potential_downs = potential_downs.sort_values('MaxLoss').reset_index(drop = True)
    potential_downs['MaxLossScore'] = potential_downs.index
    
    potential_downs = potential_downs.sort_values('upmove_to_breach').reset_index(drop = True)
    potential_downs['UpMoveScore'] = potential_downs.index
    
    potential_downs = potential_downs.sort_values('downmove_to_win').reset_index(drop = True)
    potential_downs['DownMoveScore'] = potential_downs.index
    
    potential_downs['Score'] = potential_downs.MaxLossScore + potential_downs.UpMoveScore + potential_downs.DownMoveScore
    potential_downs = potential_downs.sort_values('Score', ascending = False)
except:
    None
potential_downs

In [28]:
potential_downs[(potential_downs.MaxLoss > -10000) &
                (potential_downs.Long_Put_Strike > 2700)].sort_values('Long_Put_Strike')

,Short_Call_Strike,Long_Call_Strike,Long_Put_Strike,Delta,Gamma,Theta,Vega,time_remaining,Credit,MaxLoss,BreakEven,Maturity,ReturnOnRisk,upmove_to_breach,downmove_to_win,MaxLossScore,UpMoveScore,DownMoveScore,Score
239,"3,260.0000","3,335.0000","2,705.0000",-0.2322,0.0002,0.0034,0.0084,0.0643,34.1101,"-4,088.9900","3,294.1101",2020-03-19,0.0083,0.0032,-0.1676,43,251,239,533
238,"3,255.0000","3,335.0000","2,705.0000",-0.2443,0.0002,0.0197,0.0038,0.0643,37.1101,"-4,288.9900","3,292.1101",2020-03-19,0.0087,0.0016,-0.1676,36,183,238,457
237,"3,250.0000","3,335.0000","2,705.0000",-0.2563,0.0002,0.0357,0.0023,0.0643,40.1601,"-4,483.9900","3,290.1601",2020-03-19,0.0090,0.0001,-0.1676,32,114,237,383
240,"3,250.0000","3,335.0000","2,710.0000",-0.2569,0.0002,0.0302,0.0127,0.0643,40.0601,"-4,493.9900","3,290.0601",2020-03-19,0.0089,0.0001,-0.1661,31,107,240,378
241,"3,255.0000","3,335.0000","2,710.0000",-0.2450,0.0002,0.0143,0.0142,0.0643,37.0101,"-4,298.9900","3,292.0101",2020-03-19,0.0086,0.0016,-0.1661,35,173,241,449
242,"3,255.0000","3,335.0000","2,715.0000",-0.2457,0.0002,0.0082,0.0253,0.0643,36.9101,"-4,308.9900","3,291.9101",2020-03-19,0.0086,0.0016,-0.1645,34,195,242,471
243,"3,250.0000","3,335.0000","2,715.0000",-0.2576,0.0002,0.0242,0.0238,0.0643,39.9601,"-4,503.9900","3,289.9601",2020-03-19,0.0089,0.0001,-0.1645,30,106,243,379
244,"3,250.0000","3,335.0000","2,720.0000",-0.2585,0.0003,0.0158,0.0376,0.0643,39.8601,"-4,513.9900","3,289.8601",2020-03-19,0.0088,0.0001,-0.1630,29,105,244,378
245,"3,250.0000","3,335.0000","2,725.0000",-0.2593,0.0003,0.0078,0.0510,0.0643,39.8101,"-4,518.9900","3,289.8101",2020-03-19,0.0088,0.0001,-0.1615,28,132,245,405
247,"3,260.0000","3,340.0000","2,735.0000",-0.2518,0.0002,0.0009,0.0060,0.0643,35.3601,"-4,463.9900","3,295.3601",2020-03-19,0.0079,0.0032,-0.1584,33,258,247,538


In [65]:
ib.disconnect()